In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
%pwd


'/Users/mselmaesen/Desktop/Mole-Detection-CNN-Project_ver2/src/model'

In [3]:
model = Sequential()

 

# adding the input layer to vgg model
vgg = VGG16(weights='imagenet',include_top=False, input_shape=(64,64,3))
model.add(vgg)

model.add(Flatten())
model.add(Dense(2, activation="softmax"))
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 14,718,786
Trainable params: 14,718,786
Non-trainable params: 0
_________________________________________________________________


In [4]:
# don't train the vgg layers:
for layers in model.layers[0].layers:
    layers.trainable = False
    
# And Freeze also layer zero for good measure
model.layers[0].trainable = False

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 14,718,786
Trainable params: 4,098
Non-trainable params: 14,714,688
_________________________________________________________________


In [8]:
from keras.preprocessing.image import ImageDataGenerator

# compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# defining the constants for the model training

BATCH_SIZE = 32
EPOCHS = 20
test_path='/Users/mselmaesen/Desktop/Mole-Detection-CNN-Project_ver2/src/preprocessing/temp/test'
train_path='/Users/mselmaesen/Desktop/Mole-Detection-CNN-Project_ver2/src/preprocessing/temp/train'


In [14]:
generator = ImageDataGenerator(
    rotation_range=90,
    rescale=1/255,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    brightness_range=[0.8,1.2],
    validation_split=0.2
)
test_generator = ImageDataGenerator(
    rescale=1/255,
)


In [ ]:
train_set = generator.flow_from_directory(train_path, target_size=(64,64), batch_size=BATCH_SIZE)
test_set = test_generator.flow_from_directory(test_path, target_size=(64,64), batch_size=BATCH_SIZE)

model.fit_generator(train_set, steps_per_epoch=len(train_set.filenames)//BATCH_SIZE, epochs=10, validation_data = test_set, validation_steps=len(test_set.filenames)//BATCH_SIZE )


Found 2610 images belonging to 2 classes.
Found 290 images belonging to 2 classes.
Epoch 1/10
81/81 [==============================] - 94s 1s/step - loss: 0.5150 - accuracy: 0.7855 - val_loss: 0.4960 - val_accuracy: 0.7812
Epoch 2/10
81/81 [==============================] - 100s 1s/step - loss: 0.4746 - accuracy: 0.7901 - val_loss: 0.4690 - val_accuracy: 0.7951
Epoch 3/10
81/81 [==============================] - 92s 1s/step - loss: 0.4665 - accuracy: 0.7894 - val_loss: 0.4585 - val_accuracy: 0.8021
Epoch 4/10
41/81 [==============>...............] - ETA: 43s - loss: 0.4596 - accuracy: 0.8018

In [11]:
model.save_weights('model_transfer_learning_advenced.h5')